# Lab Assignment 4: How to Use APIs in Python
## DS 6001: Practice and Application of Data Science

### Answer Key

In this lab, you will work with the public API provided by [genius.com](https://genius.com/), a website that calls itself "the world’s biggest collection of song lyrics and musical knowledge." You will need to read the API documentation carefully, acquire an access key, and use it without sharing it to pull data from this API into Python. You will also practice using a library made specifically to wrap around `requests` to make calling from the Genius API easier.

## Problem 0
I load the libraries I need and turn off the error tracebacks to make errors easier to read:

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1
The Genius API documentation is here: https://docs.genius.com/#/getting-started-h1. Read through the documentation carefully. Although the Genius API is free and public, it still requires users an access key to use the API. In this case, Genius provides users with three codes: a client ID, a client ID secret, and a client access token. Use the documentation to find a way to obtain these codes for yourself. Write a paragraph that describes all of the steps you needed to take (but DO NOT list your access codes in this paragraph).

Some hints and cautions: 

1. Before you can use the API, you will need a regular, free user account with Genius. Sign-up here: https://genius.com/signup_or_login

2. Genius's API is built to support third-party app development, not data scientists. The language is entirely geared toward app development. Under "Authentication" there are instructions to third-party developers for guiding their own users in getting access keys. That's not relevant to getting access for yourself. It's not hard to get an access key, but the guidance here is not very clear. Be patient and read everything in the Authentication section carefully.

3. When you arrive at the page that allows you to register for API access keys, the language is still geared toward app development. You will be prompted to name your app and provide the URLs associated with the app. It doesn't much matter what you name your app, and I just used the Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for the URLs.

4. When you get your codes, copy them in a text file. In problem 2 you will copy these codes over again to a `.env` file. [4 points]

In the "Authentication" section of the API documentation, I noticed the following language in the right-hand frame:

> Access for Apps Without Users: If your application doesn't include user-specific behaviors you can use the client access token associated with your API instead of tokens for authenticated users. These tokens are only valid for read-only endpoints that are not restricted by a required scope. You can get a client access token by clicking "Generate Access Token" on the API Client management page.

My app will not have users (other than myself), so this language applies to me. This paragraph tells me I can use my own access keys. Further it tells me to click "Generate Access Token" on the API Client management page. But where is the API Client management page? I look around the documentation and see "Manage Clients" at the top of the toolbar to the left of the screen. Sure enough, that takes me to a page in which I can click on "New API Client", then I arrive at the page to name my app. I called my app "Class Example", and I placed https://collab.its.virginia.edu/portal?containerLogin=true in the next three fields. On the next page, I copy the client ID and the client secret to a text file. I click "Generate Access Token" and copy the code that appears to the same text file.

## Problem 2
Create a `.env` file for this project. Open it, copy your access codes into it, and save it. Then use Python code to load the environmental variables contained in the `.env` file, and create variables that contain each of the three codes. (You can print these variables to make sure it worked, but do not allow your access keys to display in your notebook). [4 points]

I open a console from my notebook's environoment and create a new `.env` file in my working directory by typing `!touch .env`. I then navigate to my working directory (revealing the hidden files if necessary) and open the `.env` file. I paste the keys into the file and save it. Then I load the environmental variables in the `.env` file into Python by typing:

In [2]:
dotenv.load_dotenv()

True

And I save the keys as Python variables:

In [3]:
GeniusClient = os.getenv('GeniusClient')
GeniusClientSecret = os.getenv('GeniusClientSecret')
GeniusToken = os.getenv('GeniusToken')

## Problem 3
The root for all Genius APIs is https://api.genius.com. Find the endpoint for the Search API. (You will have to click the "Authorize with Genius" button in the upper-right corner if you haven't already done so). Use the `requests` library to issue a search for Bob Dylan. Genius's API is organized in a way that every individual artist has his or her own API endpoint. Display a portion of the JSON output that displays the API endpoint path for the data on Bob Dylan. [4 points]

Hint: to authenticate, specify your access token (not your client ID or client secret) as the `access_token` parameter. You will have to dig around the JSON output to find the artist ID, but it is listed under `primary_artist` several branches down the JSON tree.

I specify the endpoint and the parameters and pass these to `requests.get`:

In [4]:
endpoint = "https://api.genius.com/search"
mypars = {'q':'Bob Dylan',
         'access_token': GeniusToken}
r = requests.get(endpoint, params=mypars)
r

<Response [200]>

The response of 200 tells me the request was successful, as far as the API is concerned. I dig through the JSON tree and find the information about Bob Dylan:

In [5]:
searchresult = json.loads(r.text)
searchresult["response"]['hits'][0]['result']['primary_artist']

{'api_path': '/artists/181',
 'header_image_url': 'https://images.genius.com/facc753d420efc53bbe1e0b63a72d70b.960x719x1.jpg',
 'id': 181,
 'image_url': 'https://images.genius.com/22306423b6ad8777d1ed5b33ad8b0d0b.1000x1000x1.jpg',
 'is_meme_verified': False,
 'is_verified': False,
 'name': 'Bob Dylan',
 'url': 'https://genius.com/artists/Bob-dylan'}

The endpoint path for the data on Bob Dylan is `/artists/181` which can be added to the end of the root address. 

## Problem 4
Add `/songs` to the end of the the endpoint path you found in problem 3 and use this path to request the 20 most popular Bob Dylan songs. Organize these data in a `pandas` data frame. [4 points]

To extract the top 20 most popular Bob Dylan songs I use the https://api.genius.com/artists/181/songs endpoint, and specify the following parameters as listed in the API documentation:

In [6]:
endpoint = "https://api.genius.com/artists/181/songs"
mypars = {'sort':'popularity',
         'access_token': GeniusToken}
r = requests.get(endpoint, params=mypars)
r

<Response [200]>

In [7]:
dylan_json = json.loads(r.text)
dylan_df = pd.json_normalize(dylan_json, record_path = ['response', 'songs'])
dylan_df

,annotation_count,api_path,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,pyongs_count,...,stats.pageviews,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url,stats.concurrents
0,127,/songs/5393247,Murder Most Foul by Bob Dylan,https://images.genius.com/7fbaf838ffdf99315408...,https://images.genius.com/7fbaf838ffdf99315408...,5393247,3360167,complete,/Bob-dylan-murder-most-foul-lyrics,35,...,439790,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
1,16,/songs/96286,We Are the World by USA For Africa,https://images.genius.com/cd231dd0e0159b8c8b25...,https://images.genius.com/cd231dd0e0159b8c8b25...,96286,4733728,complete,/Usa-for-africa-we-are-the-world-lyrics,29,...,417540,/artists/370890,https://images.genius.com/3fb2d9f68c911b547339...,370890,https://images.genius.com/3fb2d9f68c911b547339...,False,False,USA For Africa,https://genius.com/artists/Usa-for-africa,NaN
2,12,/songs/79424,Blowin' in the Wind by Bob Dylan,https://images.genius.com/afd28edb1b2d6d5d556a...,https://images.genius.com/afd28edb1b2d6d5d556a...,79424,73267,complete,/Bob-dylan-blowin-in-the-wind-lyrics,43,...,384445,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,2.0
3,1,/songs/68146,Make You Feel My Love by Adele,https://images.genius.com/e23228f25b76446cf8de...,https://images.genius.com/e23228f25b76446cf8de...,68146,82481,complete,/Adele-make-you-feel-my-love-lyrics,46,...,341581,/artists/2300,https://images.genius.com/2a44416083dd36132512...,2300,https://images.genius.com/2dfd665a4ae64d9e2353...,False,False,Adele,https://genius.com/artists/Adele,NaN
4,17,/songs/62605,The Times They Are A-Changin' by Bob Dylan,https://images.genius.com/bb0d6623d70dfad2803c...,https://images.genius.com/bb0d6623d70dfad2803c...,62605,49202,complete,/Bob-dylan-the-times-they-are-a-changin-lyrics,36,...,306366,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,2.0
5,6,/songs/103982,Knockin’ on Heaven’s Door by Guns N’ Roses (Ft...,https://images.genius.com/cb2614550098bf753ab3...,https://images.genius.com/cb2614550098bf753ab3...,103982,168784,complete,/Guns-n-roses-knockin-on-heavens-door-lyrics,26,...,295799,/artists/637,https://images.genius.com/151b716749a32d2e0d45...,637,https://images.genius.com/6a38c80ced1ad68fa66f...,False,False,Guns N’ Roses,https://genius.com/artists/Guns-n-roses,2.0
6,15,/songs/98664,All Along the Watchtower by Bob Dylan,https://images.genius.com/f727dfc7b28942c59cd6...,https://images.genius.com/f727dfc7b28942c59cd6...,98664,779,complete,/Bob-dylan-all-along-the-watchtower-lyrics,40,...,289566,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
7,19,/songs/54784,Like a Rolling Stone by Bob Dylan,https://images.genius.com/5de9704b9bd8773ddbcb...,https://images.genius.com/5de9704b9bd8773ddbcb...,54784,7,complete,/Bob-dylan-like-a-rolling-stone-lyrics,58,...,275541,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
8,8,/songs/79749,Make You Feel My Love by Bob Dylan,https://images.genius.com/cd0a26733cc459710d09...,https://images.genius.com/cd0a26733cc459710d09...,79749,5307,complete,/Bob-dylan-make-you-feel-my-love-lyrics,10,...,260681,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8

## Problem 5
Install and import the `lyricsgenius` library in Python, which is a wrapper around `requests` that works specifically with the Genius API. . Follow the guide on the GitHub repository for this library (https://github.com/johnwmillr/LyricsGenius) for instructions on using the library. Use the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan. [4 points]

In [8]:
import lyricsgenius
genius = lyricsgenius.Genius(GeniusToken)
artist = genius.search_artist("Bob Dylan", max_songs=3, sort="title")
song = genius.search_song("Tangled Up in Blue", artist.name)
song.lyrics

Searching for songs by Bob Dylan...

Song 1: "10,000 Men"
Song 2: "2 X 2"
Song 3: "32-20 Blues"

Reached user-specified song limit (3).
Done. Found 3 songs.
Searching for "Tangled Up in Blue" by Bob Dylan...
Done.


'[Verse 1]\nEarly one morning the sun was shining\nI was laying in bed\nWondering if she\'d changed at all\nIf her hair was still red\nHer folks they said our lives together\nSure was going to be rough\nThey never did like Mama\'s homemade dress\nPapa\'s bankbook wasn\'t big enough\nAnd I was standing on the side of the road\nRain falling on my shoes\nHeading out for the East Coast\nLord knows I\'ve paid some dues\nGetting through\nTangled up in blue\n\n[Verse 2]\nShe was married when we first met\nSoon to be divorced\nI helped her out of a jam, I guess\nBut I used a little too much force\nWe drove that car as far as we could\nAbandoned it out West\nSplit up on a dark sad night\nBoth agreeing it was best\nShe turned around to look at me\nAs I was walking away\nI heard her say over my shoulder\n"We\'ll meet again someday\nOn the avenue"\nTangled up in blue\n\n[Verse 3]\nI had a job in the great north woods\nWorking as a cook for a spell\nBut I never did like it all that much\nAnd one da